In [130]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 999)

In [131]:
# watch_csv = pd.read_csv('watch_data/watch_base_latest.csv')
watch_csv = pd.read_csv('watch_data_final/final 2.csv')

In [132]:
#columns we will drop because they are not visually advantageous to classify by
drop_col = [
    'brand',
    'reference',
    'name',
    'produced',
    'limited',
    'caseDiameter',
    'caseHeight',
    'caseLugWidth',
    'dialMaterial',
    'movementBrand',
    'movementCaliber',
    'movementDiameter',
    'movementJewels',
    'movementReserve',
    'movementFrequency',
    'description'
]

watch_csv.drop(drop_col, 
               axis=1,
               inplace=True
              )

In [133]:
watch_csv.drop('imagePath', axis=1, inplace=True)

In [134]:
watch_csv['caseMaterial'] = 'caseMaterial - ' + watch_csv['caseMaterial']
watch_csv['caseGlass'] = 'caseGlass - ' + watch_csv['caseGlass']
watch_csv['caseBack'] = 'caseBack - ' + watch_csv['caseBack']
watch_csv['dialColor'] = 'dialColor - ' + watch_csv['dialColor']
watch_csv['dialHands'] = 'dialHands - ' + watch_csv['dialHands']
watch_csv['caseShape'] = 'caseShape - ' + watch_csv['caseShape']
watch_csv['dialIndexes'] = 'dialIndexes - ' + watch_csv['dialIndexes']
watch_csv['movementType'] = 'movementType - ' + watch_csv['movementType']
watch_csv['movementDisplay'] = 'movementDisplay - ' + watch_csv['movementDisplay']
watch_csv['family'] = 'family - ' + watch_csv['family']
watch_csv['movementTime'] = 'movementTime - ' + watch_csv['movementTime']

In [135]:
watch_csv.head()

,family,caseMaterial,caseGlass,caseBack,caseShape,dialColor,dialIndexes,dialHands,movementType,movementDisplay,movementTime,imageLink
0,family - Pilot,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Open,caseShape - Round,dialColor - Black,dialIndexes - Arabic Numerals,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,"movementTime - Hours, Minutes, Small Seconds",https://watchbase.r.sizr.io/watch/lg/archimede...
1,family - Outdoor,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Round,dialColor - Red,dialIndexes - Arabic Numerals,dialHands - Stick,movementType - Automatic,movementDisplay - Analog,"movementTime - Hours, Minutes, Seconds",https://watchbase.r.sizr.io/watch/lg/archimede...
2,family - Engineer Hydrocarbon,caseMaterial - Titanium,caseGlass - Sapphire,caseBack - Closed,caseShape - Round,dialColor - Black,dialIndexes - Stick / Dot,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,"movementTime - Hours, Minutes, Small Seconds",https://watchbase.r.sizr.io/watch/lg/ball-watc...
3,family - Instruments,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Square,dialColor - Black,dialIndexes - Mixed,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,"movementTime - Hours, Minutes, Small Seconds",https://watchbase.r.sizr.io/watch/lg/bell-ross...
4,family - Transocean,caseMaterial - Stainless Steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Round,dialColor - Black,dialIndexes - Stick / Dot,dialHands - Stick,movementType - Automatic,movementDisplay - Analog,"movementTime - Hours, Minutes, Small Seconds",https://watchbase.r.sizr.io/watch/lg/breitling...


In [136]:
print("max length\n", len(watch_csv))

#shows the columns with NA values
has_na_vals = []
for column in watch_csv:
    check_na = watch_csv[watch_csv[column].isna()]
    if check_na.empty == False:
        has_na_vals.append([column,len(check_na)])
print("\nColumns with NA values\n", has_na_vals)

max length
 842

Columns with NA values
 [['caseMaterial', 36], ['caseGlass', 18], ['caseBack', 10], ['caseShape', 47], ['dialIndexes', 2], ['dialHands', 6], ['movementTime', 1]]


In [137]:
#we can actually input these missing labels into the dataset if we want.
#however, to speed this up, i am omitting the rows with NA data

for i in has_na_vals:
    with_na = watch_csv[watch_csv[i[0]].isna()]
    watch_csv.drop(with_na.index, axis=0, inplace=True)
#     watch_csv.reset_index(inplace=True, drop=True)

In [138]:
#clean up all special characters for all columns but image link
for column in watch_csv:
    if column != "imageLink":
        watch_csv[column] = watch_csv[column].map(lambda x: re.sub('[^A-Za-z0-9- ]+', '',x))    
        watch_csv[column] = watch_csv[column].str.slice(0,31,1)

In [139]:
watch_csv.head(1)

,family,caseMaterial,caseGlass,caseBack,caseShape,dialColor,dialIndexes,dialHands,movementType,movementDisplay,movementTime,imageLink
0,family - Pilot,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Open,caseShape - Round,dialColor - Black,dialIndexes - Arabic Numerals,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm,https://watchbase.r.sizr.io/watch/lg/archimede...


In [140]:
#'update' the new set of rows we know we need to iterate over
new_rows = []

for index, row in watch_csv.iterrows():
    new_rows.append(index)

In [141]:
#update the url link
for i in new_rows:
    splitter = ' gs://ram594/final_images/' + watch_csv.loc[i,"imageLink"].split('/')[-1]
#     splitter = 'gs://watches_draft_2/images_latest/' + watch_csv.loc[i,"imageLink"].split('/')[-1]
    watch_csv.loc[i,'imageLink'] = splitter

In [142]:
link = watch_csv['imageLink']
watch_csv.drop('imageLink', axis=1, inplace=True)
watch_csv.insert(0, 'googleLink', link)

In [143]:
#unable to diagnose issue with duplicate google links pulling same watch, so we will need to 
#cut duplicates out of the dataset
#remove any duplicates of google links

watch_csv.drop_duplicates(subset='googleLink', keep='first', inplace=True)

In [144]:
##### need to loop this maybe 10 times
#we are doing this loop to ensure that each label has at least 10 rows, otherwise autoML will deny

for i in range(0,10):
    items_count = {} #create the dictionary that will hold the columns
    for column in watch_csv:
        if column != 'googleLink':
            items_count[column] = {} #create the second dictionary to hold the count of labels
            for i in watch_csv[column]:
                if i not in items_count[column]:
                    items_count[column][i] = 1 #if label not already in dict, add
                else:
                    items_count[column][i] +=1 #else, add to the num

    #figure out what needs to be removed
    to_remove = []
    for column in items_count:
        for label in items_count[column]:
            if items_count[column][label] < 10:
                to_remove.append([column, label])

    #do the actual removing
    print("DELETE THE FOLLOWING...\n")
    for i in to_remove:
        print("NEW LINE: ",i[0], "LABEL: ", i[1])
        watch_csv = watch_csv[~watch_csv[i[0]].isin([i[1]])]

DELETE THE FOLLOWING...

NEW LINE:  family LABEL:  family - Engineer Hydrocarbon
NEW LINE:  family LABEL:  family - Crash de Cartier
NEW LINE:  caseMaterial LABEL:  caseMaterial - Yellow gold
NEW LINE:  caseMaterial LABEL:  caseMaterial - Ceramic
NEW LINE:  caseMaterial LABEL:  caseMaterial - Rose Gold
NEW LINE:  caseMaterial LABEL:  caseMaterial - Honey Gold
NEW LINE:  caseGlass LABEL:  caseGlass - Mineral
NEW LINE:  caseBack LABEL:  caseBack - Hinged
NEW LINE:  caseBack LABEL:  caseBack - Hunter
NEW LINE:  caseShape LABEL:  caseShape - Other
NEW LINE:  dialColor LABEL:  dialColor - Red
NEW LINE:  dialColor LABEL:  dialColor - Brown
NEW LINE:  dialColor LABEL:  dialColor - Cream
NEW LINE:  dialColor LABEL:  dialColor - Grey
NEW LINE:  dialColor LABEL:  dialColor - Sapphire
NEW LINE:  dialColor LABEL:  dialColor - Green
NEW LINE:  dialColor LABEL:  dialColor - Beige
NEW LINE:  dialColor LABEL:  dialColor - Paved
NEW LINE:  dialColor LABEL:  dialColor - Multi-Color
NEW LINE:  dialColor 

In [145]:
watch_csv.head()

,googleLink,family,caseMaterial,caseGlass,caseBack,caseShape,dialColor,dialIndexes,dialHands,movementType,movementDisplay,movementTime
0,gs://ram594/final_images/ua7939-c1-3-66.jpg,family - Pilot,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Open,caseShape - Round,dialColor - Black,dialIndexes - Arabic Numerals,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm
3,gs://ram594/final_images/br0194brocket-e7.jpg,family - Instruments,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Square,dialColor - Black,dialIndexes - Mixed,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm
4,gs://ram594/final_images/a1931012-bb68-433x-a...,family - Transocean,caseMaterial - Stainless Steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Round,dialColor - Black,dialIndexes - Stick Dot,dialHands - Stick,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm
6,gs://ram594/final_images/a1036012ba91154a-a1.jpg,family - Transocean,caseMaterial - Stainless Steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Round,dialColor - Black,dialIndexes - Stick Dot,dialHands - Stick,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Se
8,gs://ram594/final_images/ua7939-c1-1-sw-df.jpg,family - Pilot,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Open,caseShape - Round,dialColor - Black,dialIndexes - Arabic Numerals,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm


In [146]:
watch_csv.to_csv(index=False, path_or_buf='watch_data_final/watch_csv.csv')

In [147]:
watch_csv.index

Int64Index([  0,   3,   4,   6,   8,   9,  10,  11,  12,  15,
            ...
            823, 824, 826, 827, 828, 830, 831, 834, 835, 841],
           dtype='int64', length=404)

## FINDING COSINE SIMILARITY FOR RECOMMENDATIONS

In [148]:
#make a new copy df 
cos_df = watch_csv.copy(deep=True)

In [149]:
#we arent going to use google link to find similarity, so drop it
cos_df.drop('googleLink', axis=1, inplace=True)

In [150]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [151]:
#have the watches as their set of labels and then make them a string for comparison
watches = []

cols = cos_df.columns

for row in range(0,len(cos_df.index)): #go through the rows
    watch = ""
    for column in cols:
        watch = watch + cos_df.iloc[row][column] + " "
    watches.append(watch)
    

In [152]:
cos_df.head()

,family,caseMaterial,caseGlass,caseBack,caseShape,dialColor,dialIndexes,dialHands,movementType,movementDisplay,movementTime
0,family - Pilot,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Open,caseShape - Round,dialColor - Black,dialIndexes - Arabic Numerals,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm
3,family - Instruments,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Square,dialColor - Black,dialIndexes - Mixed,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm
4,family - Transocean,caseMaterial - Stainless Steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Round,dialColor - Black,dialIndexes - Stick Dot,dialHands - Stick,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm
6,family - Transocean,caseMaterial - Stainless Steel,caseGlass - Sapphire,caseBack - Closed,caseShape - Round,dialColor - Black,dialIndexes - Stick Dot,dialHands - Stick,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Se
8,family - Pilot,caseMaterial - Stainless steel,caseGlass - Sapphire,caseBack - Open,caseShape - Round,dialColor - Black,dialIndexes - Arabic Numerals,dialHands - Sword,movementType - Automatic,movementDisplay - Analog,movementTime - Hours Minutes Sm


In [153]:
#example
watches[0]

'family - Pilot caseMaterial - Stainless steel caseGlass - Sapphire caseBack - Open caseShape - Round dialColor - Black dialIndexes - Arabic Numerals dialHands - Sword movementType - Automatic movementDisplay - Analog movementTime - Hours Minutes Sm '

In [154]:
#we are using tfidf vectorizer to find the count of words in each set of watch labels
tvec = TfidfVectorizer(stop_words='english')
sparse_matrix = tvec.fit_transform(watches)

In [155]:
#this helps visualize the actual term frequency that we are looking at 
#using tfidf, we are accounting for frequency across all watches
doc_frequency_matrix = sparse_matrix.todense()

df = pd.DataFrame(
    doc_frequency_matrix,
    columns = tvec.get_feature_names(),    
    index = watches
)

In [156]:
#example
df.head(1)

,ad,analog,arabic,audemars,automatic,baton,black,blue,caseback,caseglass,casematerial,caseshape,closed,dauphine,dial,dialcolor,dialhands,dialindexes,digital,dot,double,family,feuille,gold,handwound,hours,instruments,jules,minutes,mixed,movementdisplay,movementtime,movementtype,numerals,open,outdoor,pilot,pink,processor,proprietary,quartz,rectangular,red,reverso,round,sapphire,se,silver,sm,square,stainless,steel,stick,sword,transocean,watch,white
family - Pilot caseMaterial - Stainless steel caseGlass - Sapphire caseBack - Open caseShape - Round dialColor - Black dialIndexes - Arabic Numerals dialHands - Sword movementType - Automatic movementDisplay - Analog movementTime - Hours Minutes Sm,0.0,0.124295,0.277569,0.0,0.156584,0.0,0.2065,0.0,0.118597,0.118597,0.118597,0.118597,0.0,0.0,0.0,0.118597,0.118597,0.118597,0.0,0.0,0.0,0.118597,0.0,0.0,0.0,0.118597,0.0,0.0,0.118597,0.0,0.118597,0.118597,0.118597,0.277569,0.280974,0.0,0.517654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1689,0.118597,0.0,0.0,0.235605,0.0,0.156181,0.156181,0.0,0.294369,0.0,0.0,0.0


In [157]:
print(cosine_similarity(df,df))

[[1.         0.45760378 0.52566919 ... 0.41219458 0.35761888 0.60233595]
 [0.45760378 1.         0.51186064 ... 0.4014182  0.34835555 0.45770073]
 [0.52566919 0.51186064 1.         ... 0.81396696 0.75628867 0.91641469]
 ...
 [0.41219458 0.4014182  0.81396696 ... 1.         0.60623871 0.74318546]
 [0.35761888 0.34835555 0.75628867 ... 0.60623871 1.         0.68608138]
 [0.60233595 0.45770073 0.91641469 ... 0.74318546 0.68608138 1.        ]]


## WORKING ON ACTUAL TFIDF FROM NEW SAMPLE WHEN IT IS GIVEN

### The code below is an example watch. the code under it is functions to assist

In [158]:
#transforms the labels into tfidf values
#returns a df of the tfidf values for each word
def get_tfidf_of_watch(labels:list):
    test_matrix = tvec.transform(test_watch)

    test_frequency_matrix = test_matrix.todense()

    test_df = pd.DataFrame(
        test_frequency_matrix,
        columns = tvec.get_feature_names(),    
        index = test_watch
    )
    return test_df
#     print(df)

In [159]:
#this gets us the most similar watches by cosine similarity and their indices in 'cos_df' 
def find_indexes_of_similar(cos_sim):
    
    comparison_list = []
    
    for i in range(0, len(cos_sim)):
        comparison_list.append((cos_sim[i], i))
    comparison_list.sort(reverse=True)
    
    return comparison_list
        

In [160]:
#grab the top n watches based on similar labels
def get_top_n_recommendations_indices(comparison_list, n):
    recommended_indices = []
    recommended_watches = []
    i = 0

    while (i < n):
        index = comparison_list[i][1] #get the index of the watch we will refer to
        
        #if we are looking at ourselves as a watch, ignore it and go to another one
        if index == 0:
            n = n + 1
            i = i + 1
            continue
        
        recommended_indices.append(index)
        i = i + 1
    
    original_csv = pd.read_csv('watch_data_final/final 2.csv')

    for i in recommended_indices:
        recommended_watches.append(original_csv.iloc[i]['name'])
        
    return recommended_watches

In [161]:
#an example about how the watch will work when it gets received 
test_watch = [watches[0]]
print("Watch labs:", test_watch,"\n")

#returns a df of the tfidf values for each word
test_tfidf = get_tfidf_of_watch(test_watch)

#get the cosine similarity between the new watch and all of our watches in the database
cos_sim = cosine_similarity(df, test_tfidf)

#this gets us the most similar watches by cosine similarity and their indices in 'cos_df' 
comparison_list = find_indexes_of_similar(cos_sim)

#grab the top 5 watches based on similar labels
recommended = get_top_n_recommendations_indices(comparison_list,5)

#this shows us our recommended
# print("\n\n", recommended)
for i in recommended:
    print("Recommended watch:",i)

Watch labs: ['family - Pilot caseMaterial - Stainless steel caseGlass - Sapphire caseBack - Open caseShape - Round dialColor - Black dialIndexes - Arabic Numerals dialHands - Sword movementType - Automatic movementDisplay - Analog movementTime - Hours Minutes Sm '] 

Recommended watch: Reverso Lady Quartz
Recommended watch: Reverso Classic Large Duoface Small Seconds Stainless Steel / Silver
Recommended watch: Reverso Classic Large Small Seconds Stainless Steel / Silver / Fagliano
Recommended watch: Transocean Chronograph 1461 Stainless Steel / Black / Calf / Pin
Recommended watch: Reverso Tribute Moon Stainless Steel / Silver
